## JTAG Tests: Glitch Sensitivity

This requires the following hardware setup:

* CW-Husky, SAM4S2AA Target, CW313 board
* 20-pin cable from front panel (UserIO) to "JTAG" header, OR use the mixer board (mixer board has series resistors on TDI/TDO which seemed to result in better high-frequency results)
* 20-pin cable from ChipWhisperer connector (side panel on CW) to target board
* No SMA cable connected
* HS1 jumper to use clock sync
* Trigger jumper routing GPIO4 to trigger (default for CW313 / ChipWhisperer)

These tests were run with the jumper mounted on HDR1, which shorts out the shunt resistor. This reduces the shunt resistor signal (but does not eliminate it fully).

In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "data_store"

In [ ]:
def capture_ttest_jtag(N, splot=None, repeat=1, resync=True, xorinout=False):
    """Captures a T-Test (Modified for Logic Analyzer Block Instead)"""
    import numpy as np
    from tqdm.autonotebook import trange
    ktp = cw.ktp.TVLATTest()
    ktp.init(N)

    group1 = []
    group2 = []
    g1_i = 0
    g2_i = 0
    for i in trange(2*N, desc='Capturing traces'):
        key, text = ktp.next()  # TVLA T-Test changes PT between two options
        
        temp_wave = []
        for j in range(0, repeat):

            scope.LA.arm()

            trace = cw.capture_trace(scope, target, text, key)


            if trace is None:
                print("Serial Error - dropped trace")
                continue

            raw = scope.LA.read_capture_data()
            tck = scope.LA.extract(raw, 5)
            tdo = scope.LA.extract(raw, 3)
            tdi = scope.LA.extract(raw, 7)
            tms = scope.LA.extract(raw, 6)

            wave = tdo
            
            if xorinout:
                wave = wave ^ tdi
            #wave = trace.wave

            #Deal with phase offset issue
            if resync:
                if tdi[0] == 0:
                    wave = wave[0:-1]
                else:
                    wave = wave[1:]
                
            temp_wave.append(wave)
        
        wave = np.mean(temp_wave, axis=0)
            
        if i % 25 == 0:
            if splot:
                splot.update(wave) # wave is the name for the data for our power trace
            
     
        if trace.textin == bytearray([0xda, 0x39, 0xa3, 0xee, 0x5e, 0x6b, 0x4b, 0x0d, 0x32, 0x55, 0xbf, 0xef, 0x95, 0x60, 0x18, 0x90]):
            group1.append(wave)
            g1_i += 1
        else:
            group2.append(wave)
            g2_i += 1

    group1 = np.array(group1)
    print(len(group1))
    group2 = np.array(group2)
    print(len(group2))
    
    return (group1, group2)

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
scope = cw.scope(bitstream = r"firmwares/cwhusky_top_latriggeredwithoffset.bit")
target = cw.target(scope, cw.targets.SimpleSerial2)

In [ ]:
scope.default_setup()

In [ ]:
def jtag_clkout(enabled):
    if enabled:
        data = 0x08
    else:
        data = 0x00

    CODE_READ = 0x80
    CODE_WRITE = 0xC0
    scope.userio.oa.sendMessage(CODE_WRITE, "USERIO_DEBUG_DRIVEN", [data])
    
    # Can use this to check write worked
    #scope.userio.oa.sendMessage(CODE_READ, "USERIO_DEBUG_DRIVEN")

def jtag_glitchout(enabled):
    if enabled:
        data = 0x10
    else:
        data = 0x00

    CODE_READ = 0x80
    CODE_WRITE = 0xC0
    scope.userio.oa.sendMessage(CODE_WRITE, "USERIO_DEBUG_DRIVEN", [data])
    
    # Can use this to check write worked
    #scope.userio.oa.sendMessage(CODE_READ, "USERIO_DEBUG_DRIVEN")
    

In [ ]:
jtag_clkout(False)

## Build Firmware - 15 MHz Internal Clock, External Output (7.5 MHz) for sync

In [ ]:
# program firmware onto target
prog = cw.programmers.SAM4SProgrammer
cw.program_target(scope, prog, "firmwares/fw-15mhz-clkout.hex")

In [ ]:
import time
scope.io.target_pwr = False
time.sleep(0.5)
scope.io.target_pwr = True

In [ ]:
# Baud is lower so it works from internal oscillator which isn't as precise
target.baud = 38400

In [ ]:
# When device is running at 2 MHz baud calculation is off - measuring actual baud shows this is correct:
# Uncomment this when compiling for 2 MHz firmware
#target.baud = 62750

In [ ]:
#We don't use the clock output
scope.io.hs2 = None

In [ ]:
# We don't care about the ADC sampling rate, set this low.
# Should actually turn off the ADC as we are way overclocking it in some cells.
# But I think it's fine?
scope.clock.adc_mul = 1

In [ ]:
scope.clock.clkgen_src = "extclk"
scope.clock.pll.set_outfreqs(7.5E6, 221E6, 1, force_recalc=True)
#scope.clock.pll.set_outfreqs(7.5E6, 50E6, 1, force_recalc=True)

In [ ]:
#scope.clock.clkgen_freq = 10E6
#scope.clock.adc_mul = 1

## JTAG Function Definitions

It's assumed you've already seen the previous JTAG notebooks, so these will not be described here.

In [ ]:
def read_tdo_status():
    pins = scope.userio.status
    if pins & (1<<3):
        return True
    else:
        return False
    
def write(tms, tdi):
    old = scope.userio.drive_data
    old &= ~(1<<6 | 1<<7)
    if tms:
        old |= 1<<6
    if tdi:
        old |= 1<<7
    
    scope.userio.drive_data = old
    scope.userio.drive_data = old | (1<<5)
    scope.userio.drive_data = old & ~(1<<5)

In [ ]:
def setup_bypass(verbose=True):
    #Take control of TDI, TMS, TCK
    scope.userio.direction = 0b11100000
    
    write(1, 1)
    write(1, 1)
    write(1, 1)
    write(1, 1)
    write(1, 1)

    write(0, 1) #
    write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    #Send a bunch of 1's to force bypass mode
    for i in range(0, 10):
        write(0, 1)

    #exit shift-IR state
    write(1,1)

    write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    for i in range(0, 10):
        write(0, 0)

    tdo_result = []

    for i in range(0, 10):
        tdo_result.append(read_tdo_status())
        if i == 0:
            write(0, 1)
        else:
            write(0, 0)

    if tdo_result[0:10] == [False, True, False, False, False, False, False, False, False, False]:
        if verbose:
            print("JTAG Setup successful - bypass mode enabled, saw '1' sequence successfuly")
        return True
    else:
        if verbose:
            print("JTAG Setup not successful")
            print(tdo_result)
        return False

## Logic Analyzer Setup

In [ ]:
help(scope.LA)
print(scope.LA.max_capture_depth)

In [ ]:
assert scope.LA.present, 'There is no logic analyzer in this FPGA bitfile!'

In [ ]:
import time

scope.LA.enabled = False
scope.clock.clkgen_freq = 200E6
scope.clock.reset_dcms()
#scope.LA.enabled = True
#scope.LA.clk_source = "pll"
#scope.LA.reset_MMCM()
#scope.clock.pll._warning_freq = 400E6

scope.LA.enabled = True
scope.LA._warning_frequency = 401e6 # prevent warning message that we're setting the sampling clock too high
scope.LA.oversampling_factor = 1
scope.LA.downsample = 1
scope.LA.capture_group = 'USERIO 20-pin'
scope.LA.trigger_source = "capture"
scope.LA.capture_depth = 16376

In [ ]:
scope.LA.oversampling_factor = 1
scope.LA.downsample = 1
scope.LA.capture_group = 'USERIO 20-pin'
scope.LA.trigger_source =  "capture"
scope.LA.capture_depth = 16376

max_freq = 0
for i in range(25, 400, 5):
    scope.clock.clkgen_freq = i * 1E6
    scope.clock.reset_dcms()
    #scope.clock.recal_pll()
    #scope.clock.reset_dcms()
    time.sleep(0.05)
    scope.LA.reset_MMCM()
    time.sleep(0.05)
    if scope.LA.locked == False:
        max_freq = i
        print("Logic Analyzer failed at {:d} MHz".format(i))
        break
scope.clock.clkgen_freq = 20E6

### Testing Capture & Plotting

In [ ]:
scope.clock.clkgen_freq = 50E6

In [ ]:
scope.LA.reset_MMCM()

In [ ]:
assert scope.LA.locked

In [ ]:
#Turn on the output clock - we won't see feedback yet as haven't turned on JTAG bypass mode
jtag_clkout(True)
scope.userio.direction = 0b11100000

In [ ]:
scope.LA.arm()

In [ ]:
scope.LA.trigger_now()

In [ ]:
scope.LA.errors

On the standard JTAG pinout, we have:

* d[2] = nrst
* d[3] = tdo
* d[4] = rclk
* d[5] = tck
* d[6] = tms
* d[7] = tdi

In [ ]:
raw = scope.LA.read_capture_data()
tck = scope.LA.extract(raw, 5)
tdo = scope.LA.extract(raw, 3)
tdi = scope.LA.extract(raw, 7)
tms = scope.LA.extract(raw, 6)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(tck))
O1 = o.line(xrange, tck + 6, line_color='black')
O2 = o.line(xrange, tdo + 4, line_color='blue')
O3 = o.line(xrange, tdi + 2, line_color='red')

legend = Legend(items=[
    LegendItem(label='TCK', renderers=[O1]),
    LegendItem(label='TDO', renderers=[O2]),
    LegendItem(label='TDI', renderers=[O3]),
])
o.add_layout(legend)

In [ ]:
show(o)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=600)

xrange = range(25)
O3 = o.line(xrange, tdi[0:25] + 2, line_color='red')

legend = Legend(items=[
    LegendItem(label='TDI', renderers=[O3]),
])
o.add_layout(legend)

In [ ]:
show(o)

In [ ]:
tdi[0:25]

What you expect from the above is that TDI (input to the device) is seen oscillating between 1 & 0. The sampling is done on the TCK clock so we won't see anything interesting on that line (will show as always high).

![](docs/jtag_tdo_10mhz.png)

The TDI output will look like this:
`array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])`

With that working, let's turn on the TDI->TDO path and figure out how close to the sun we can fly.

### Testing JTAG Feedthrough

Watch the following says `JTAG Setup successful - bypass mode enabled, saw '1' sequence successfuly`.

In [ ]:
jtag_clkout(False)
setup_bypass()
jtag_clkout(True)

Alright, now we can do a new capture:

In [ ]:
assert scope.LA.locked

In [ ]:
scope.LA.arm()
scope.LA.trigger_now()
time.sleep(0.1)

In [ ]:
print(scope.LA.errors)
raw = scope.LA.read_capture_data()
tck = scope.LA.extract(raw, 5)
tdo = scope.LA.extract(raw, 3)
tdi = scope.LA.extract(raw, 7)
tms = scope.LA.extract(raw, 6)

In [ ]:
print(tdo)
print(tdi)

These should all match up - you may get a phase shift so you'll see the 1/0 offset.
```
[0 1 0 ... 1 0 1]
[0 1 0 ... 1 0 1]
```
We can compare all of the inputs & outputs to check they are matching, either directly OR with an offset of one cycle:

In [ ]:
print(np.all(tdi == tdo))
print(np.all(tdi[0:-1] == tdo[1:]))

In [ ]:
scope.clock.pll._warning_freq = 400E6

In [ ]:
scope.LA

### Checking Maximum LA Lock Frequency

In [ ]:
import time

max_freq = 300
for i in range(25, 400, 5):
    scope.clock.clkgen_freq = i * 1E6
    scope.clock.reset_dcms()
    time.sleep(0.05)
    scope.LA.reset_MMCM()
    time.sleep(0.05)
    if scope.LA.locked == False:
        max_freq = i
        print("Logic Analyzer failed at {:d} MHz".format(i))
        #break

#Keep it cool!
scope.clock.clkgen_freq = 10E6

This is the maximum frequency the logic analyzer worked up to. This will vary per unit, with FPGA build, and with temperature. Going beyond the value determined above will just be useless, as you won't see the values on our scope. You can check the unit temperature with:

In [ ]:
scope.XADC

### Checking Maximum TDI->TDO Frequency

The following will test the TDI->TDO path and will help see how well the received data matches the expected 1/0 split.

In [ ]:
max_freq = 300
frequency_sweep_list = []
try:
    for i in range(50, max_freq, 1):
        scope.clock.clkgen_freq = i * 1E6
        time.sleep(0.01)
        scope.LA.reset_MMCM()
        time.sleep(0.01)
        assert scope.LA.locked

        scope.LA.arm()
        scope.LA.trigger_now()
        time.sleep(0.1)
        raw = scope.LA.read_capture_data()
        tdo = scope.LA.extract(raw, 3)
        tdi = scope.LA.extract(raw, 7)

        result = tdo ^ tdi
        
        zeros = np.sum(result == 0)
        ones = np.sum(result == 1)

        print("{:d} zeros, {:d} ones at {:d} MHz".format(zeros, ones, i))
        frequency_sweep_list.append( (zeros, ones, i)  )
except:
    scope.clock.clkgen_freq = 10E6
    raise

In [ ]:
np.save("jtag_frequency_sweep_april42024_withdecouplingcaps_newLAbuild_xorout.npy", frequency_sweep_list)

Due to phase shifts you'll see various results that are "naturally" faulty, as the S&H times at the TCK/TDI input are being violated. Ideally we'd like to see such violations near the upper end of the sampling frequency.

```
8188 zeros, 8188 ones at 211 MHz
8188 zeros, 8188 ones at 212 MHz
8188 zeros, 8188 ones at 213 MHz
8188 zeros, 8188 ones at 214 MHz
8188 zeros, 8188 ones at 215 MHz
8188 zeros, 8188 ones at 216 MHz
8188 zeros, 8188 ones at 217 MHz
8188 zeros, 8188 ones at 218 MHz
8185 zeros, 8191 ones at 219 MHz
8067 zeros, 8309 ones at 220 MHz
7665 zeros, 8711 ones at 221 MHz
7755 zeros, 8621 ones at 222 MHz
8076 zeros, 8300 ones at 223 MHz
8477 zeros, 7899 ones at 224 MHz
8915 zeros, 7461 ones at 225 MHz
9573 zeros, 6803 ones at 226 MHz
2465 zeros, 13911 ones at 227 MHz
0 zeros, 16376 ones at 228 MHz
0 zeros, 16376 ones at 229 MHz
16376 zeros, 0 ones at 230 MHz
0 zeros, 16376 ones at 231 MHz
0 zeros, 16376 ones at 232 MHz
16376 zeros, 0 ones at 233 MHz
```


In [ ]:
# If you'd like to load an existing list, just run this:
frequency_sweep_list = np.load("jtag_frequency_sweep.npy")

## T-Testing

In [ ]:
jtag_clkout(True)

In [ ]:
scope.clock.pll.set_outfreqs(7.5E6, 159E6, 1, force_recalc=True)
#scope.LA.reset_MMCM()

In [ ]:
print(scope.clock)
print(scope.adc)
print(scope.LA)

In [ ]:
#Setting an offset here will sweep through
#scope.adc.offset = 6500

#Set to 10K offset for 180MS/s
scope.adc.offset = 10000

#Set to 10K for 158MS/s
#scope.adc.offset = 10000

#Set to 8K for 109MS/s
#scope.adc.offset = 5000

# Samples doesn't matter, but code downloads data by default, so keep this lower to avoid wasting too much time.
# Should probably avoid downloading ADC data eventually since it's not used at all.
scope.adc.samples = 1000

In [ ]:
scope.clock.extclk_error = False

In [ ]:
N = 10000
group1, group2 = capture_ttest_jtag(N, repeat=1, resync=False, xorinout=True)

**IMPORTANT**: If the above capture isn't working & you reset the device, be sure you also configure the JTAG bypass mode to be enabled again!

In [ ]:
#nptsave("jtag_offset0_clock221E6_10k", group1, group2)
#nptsave("jtag_offset15k_clock221E6_10k", group1, group2)
#nptsave("jtag_offset30k_clock221E6_10k", group1, group2)
#nptsave("jtag_offset6500_clock221E6_10_25avgpertrace", group1, group2)


#nptsave("jtag_offset6500_clock221E6_100_25avgpertrace", group1, group2)
#nptsave("jtag_offset6500_clock221E6_2500_1avgpertrace", group1, group2)


#The following resulted in interesting results, but didn't align?
#nptsave("jtag_offset6500_clock221E6_2500_1avgpertrace_run2", group11, group12)
#nptsave("jtag_offset6500_clock221E6_2500_1avgpertrace_run2", group11, group12)

#The following had results near point 11xxxx
#nptsave("jtag_offset6500_clock221E6_10k_avg_noresync_extclock", group1, group2)
#nptsave("jtag_offset6500_clock221E6_10k_avg_resync_extclock", group1, group2)

#nptsave("jtag_offset6500_clock221E6_200k_1avg_xor_extclock", group1, group2)

#April 3, 2024:
#nptsave("jtag_offset10000_clock158E6_10k_1avg_xor_extclock", group1, group2)
#nptsave("jtag_offset12000_clock178E6_10k_1avg_xor_extclock", group1, group2)
#nptsave("jtag_offset10000_clock186E6_10k_xor_extclock", group1, group2)
#nptsave("jtag_offset10000_clock159E6_10k_xor_extclock", group1, group2)


#April 5, 2024
#nptsave("jtag_offset10000_clock159E6_10k_xor_extclock_new", group1, group2)

In [ ]:
#group1, group2, N = nptload("jtag_offset10000_clock158E6_10k_1avg_xor_extclock")

In [ ]:
import numpy as np
mean1 = np.mean(group1, axis=0)
mean2 = np.mean(group2, axis=0)
cw.plot(mean2) * cw.plot(mean1)

In [ ]:
#from scipy import signal
#sos = signal.butter(5, 0.05, 'highpass', output='sos')

#group11 = signal.sosfilt(sos, group1)
#group12 = signal.sosfilt(sos, group2)

In [ ]:
from scipy.stats import ttest_ind
M = int(N/2)
t1_val = ttest_ind(group1[:M], group2[:M], axis=0, equal_var=False)[0]
t2_val = ttest_ind(group1[M:], group2[M:], axis=0, equal_var=False)[0]

#t1_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]
#t2_val = ttest_ind(group11, group22, axis=0, equal_var=False)[0]

In [ ]:
%matplotlib notebook
plot_t(t1_val, M, "JTAG Glitch Experiments")
plot_t(t2_val, M, "JTAG Glitch Experiments")

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]

In [ ]:
plt.figure()
plot_t(t_val, N, "JTAG Glitch Experiments")

In [ ]:
# Plotting glitches over time

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
plt.plot(np.sum(group1, axis=0))

In [ ]:
scope.clock.pll.set_outfreqs(7.5E6, 159E6, 1, force_recalc=True)

### Additional JTAG Speed Tests

In [ ]:
for i in range(1490, 1520, 5):
    scope.clock.pll.set_outfreqs(7.5E6, i * 1E5, 1, force_recalc=True)
    time.sleep(0.1)
    scope.LA.reset_MMCM()
    time.sleep(0.5)
    #assert scope.LA.locked
    
    scope.io.nrst = False
    time.sleep(0.1)
    scope.io.nrst = None
    time.sleep(0.1)
    print(target.read())
    
    print("*** Attempting {:d}".format(i))
    
    try:
        capture_ttest_jtag(1, repeat=1, resync=False, xorinout=True)   
        print(">>> Worked @ {:d}".format(i))
        break
    except:
        pass
    
    try:
        capture_ttest_jtag(1, repeat=1, resync=False, xorinout=True)   
        print(">>> Worked @ {:d}".format(i))
        break
    except:
        pass
   
    scope.io.nrst = False
    time.sleep(0.1)
    scope.io.nrst = None
    time.sleep(0.1)
    print(target.read())

    try:
        capture_ttest_jtag(4, repeat=1, resync=False, xorinout=True)   
        print(">>> Worked @ {:d}".format(i))
        break
    except:
        pass
        
scope.io.nrst = None

### CPA Capture

In [ ]:
scope.LA

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 500000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    scope.LA.arm()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    raw = scope.LA.read_capture_data()
    tck = scope.LA.extract(raw, 5)
    tdo = scope.LA.extract(raw, 3)
    tdi = scope.LA.extract(raw, 7)
    tms = scope.LA.extract(raw, 6)

    wave = tdo
            
    wave = wave ^ tdi  
    
    #wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 100 == 0:
        splot.update(wave) # wave is the name for the data for our power trace
        
    if i % 10000 == 0:
        try:
            print("Attempting save at {:d}".format(i))
            #raise IOError("Change 'False' to True and run this again")
            save_ets(waves, textins, textouts, keys, "d:/data_store/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_k.ets", overwrite=False)
            print("phew")
        except:
            print("Hmm... save failed, skipped!")
            pass

In [ ]:
wave

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_60k.ets", overwrite=True)

### Shunt Resistor SCA Comparison

We'll use the same sampling frequency as above to do a shunt resistor measurement. Add a SMA cable between the shunt output and the ChipWhisperer Husky input.

We'll turn OFF the JTAG clock while doing this test to reduce the noise. If you want a better comparison (better in terms of will more accurately match what is recorded above), be sure `jtag_clkout(True)` is set instead.

In [ ]:
jtag_clkout(False)

Record a lot of samples since we don't know where leakage will occur:

In [ ]:
scope.adc.samples = 100000

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
N = 5000
group1, group2 = capture_ttest(N, splot=splot)

In [ ]:
#nptsave("jtagglitch_221ms_5k_sca", group1, group2)

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]

In [ ]:
%matplotlib notebook
plot_t(t_val, N, "T-Test, SCA (for JTAG Comparison)")

In [ ]:
%matplotlib notebook
plot_t(t_val[6500:(6500+16000)], N, "T-Test, SCA (for JTAG Comparison)")

In [ ]:
#np.save("jtagglitch_221ms_5k_scashunt", t_val)

### Shunt resistor CPA Comparison

Again - set the JTAG clock as being on or off as required.

In [ ]:
jtag_clkout(False)

In [ ]:
#scope.clock.clkgen_freq = 221E6

In [ ]:
scope.clock.pll.set_outfreqs(7.5E6, 158E6, 1, force_recalc=True)

In [ ]:
scope.adc.offset = 0
scope.adc.samples = 100000

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 5000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    #scope.LA.arm()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue

    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 5 == 0:
        splot.update(wave) # wave is the name for the data for our power trace

In [ ]:
scope.adc.trig_count / 10

In [ ]:
cw.plot(np.mean(waves, axis=0))

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/jtagglitch_15MHzCPU_158MHz_shuntcpa.ets", overwrite=True)

In [ ]:
def get_min_max2(results, knownkey, byteindex=0):
    max1 = np.max(results[0:knownkey], axis=0)
    max2 = np.max(results[(knownkey+1):], axis=0)
    maxline = np.max(np.array([max1, max2]), axis=0)
    
    min1 = np.min(results[0:knownkey], axis=0)
    min2 = np.min(results[(knownkey+1):], axis=0)
    minline = np.min(np.array([min1, min2]), axis=0)
    return minline, maxline

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
fig, ax = plt.subplots(figsize=(8, 4))
#ax.plot(results2[0,1,0,:,:].T, color='gray')

minline, maxline = get_min_max2(newresults, dataset["0/0/key"][0,plotkey], plotkey)
ax.plot(minline, color='gray')
ax.plot(maxline, color='gray')

ax.plot(newresults[dataset["0/0/key"][0,plotkey]], color='red')
#ax.plot(results2[0,1,0,dataset["0/0/key"][0,1],:].T, color='green')
#ax.plot(results2[0,2,0,dataset["0/0/key"][0,2],:].T, color='blue')
#ax.plot(results2[0,2,0,dataset["0/0/key"][0,15],:].T, color='orange')
ax.set_xlabel('Samples')
ax.set_ylabel('CPA')
plt.show()